# Assignment Week 3 (Part 1)

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries imported!')

Libraries imported!


## 1. Create the dataframe 

### Download data


The data is stored in the df variable

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print('Data downloaded!')

Data downloaded!


Save the data in a pandas DataFrame

In [6]:
columns_name = ['Postcode', 'Borough','Neighborhood']
df_data = pd.DataFrame(columns = columns_name)
df_data= df[0]

#Replace "Not assigned" strings values with numpy NaN values and get the first five rows
df_data['Borough'].replace({'Not assigned': np.nan}, inplace = True)

#Drop the NaN values 
df_data.dropna(inplace = True)

#Combined rows into one row in the "Neighborhood" column, separated with a comma
df_data_2 = df_data.groupby("Postcode").agg(lambda x:','.join(set(x)))

#Assign the borough values to the neighborhood where a cell has a borough but a Not assigned 
#neighborhood and get the first five values with pd.head()
df_data_2.loc[df_data_2['Neighborhood']=="Not assigned",'Neighborhood']=df_data_2.loc[df_data_2['Neighborhood']=="Not assigned",'Borough']
df_data_2.head()

,Borough,Neighborhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill"
M1E,Scarborough,"West Hill,Morningside,Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


## 2. Get longitude and latitude of Toronto, Ontario and it's Neigborhood


Download and import csv file

In [7]:
df_latlog = pd.read_csv("/home/yan/Descargas/Geospatial_Coordinates.csv")
print('Download data')

Download data


In [8]:
#Set the longitude and latitude columns in the dataframe
df_data_2['Latitude'] = df_latlog.Latitude.values
df_data_2['Longitude'] = df_latlog.Longitude.values
df_data_2.head()

,Borough,Neighborhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill",43.784535,-79.160497
M1E,Scarborough,"West Hill,Morningside,Guildwood",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Make a new dataframe with only the Borough of Toronto

In [9]:
Toronto_data = df_data_2[df_data_2['Borough'].str.contains("Toronto")].reset_index(drop=True)
Toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,East Toronto,"India Bazaar,The Beaches West",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


Get the longitude and latitude of Toronto

In [10]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


create map of Toronto using latitude and longitude values

In [ ]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto